In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.python.keras import backend as K
import tensorflow as tf
import collections
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from pandas import ExcelWriter
import pandas.io.formats.excel
from funciones import ErrorAuno,ErrorAcero,ErrorPosicionInicial,TensorUpdatePosicionInicial,TensorUpdateCiclo

from tensorflow.keras.layers import (Activation, AveragePooling2D, BatchNormalization, Cropping2D,
                                     Concatenate, Conv1D, Conv2D, Dense, DepthwiseConv2D, Dropout,
                                     Embedding, Flatten, GlobalAveragePooling2D, Lambda, MaxPool2D,
                                     MaxPooling1D, ReLU, Reshape, ZeroPadding2D)

In [7]:
##### Funciones para trabajar cada caso según el tipo de error si el mismo está dado por la escritura de un 0 cuando
##### debía ser un 1(ErrorAcero) o viceversa. Se ha desarrollado un algoritmo que permite colocar luego del primer bit con
####error encontrado en el resto de las posiciones a 0 o a unos : Se analizan los valores comenzando por el BMS(Bit más
#signif)comparando el valor con el error y el valor que se debe escribir correcto donde haya un cambio de valor se analiza
#### si el cambio es de un 0 x 1 se coloca el resto de los bits a 0, y luego se inyectan los demás errores y si el cambio
#### es de un 1 x 0 se colocan los bits siguientes a 1 y se mantienen los errores restantes haciendo un Bitwise con los
#valores  generados por errores estaticos a 1 o a 0

In [9]:
# Conversión del tensor generado por la red a naturales: para ello se multiplica el número generado por la red que es un 
#un número real x 2**bit de la parte fraccionaria colocando el valor en un entero de 32 bits(de esta forma se convierten a 
# enteros) luego para llevarlo a naturales(a los valores negativos se les suma 2**al tamaño de la palabra que en este caso
# es de 16 bits)

## Declaración de variables y preparación de los datos que entran al ciclo

In [ ]:
#values_to_run
#mask
#v_m
#v_b
#mask_0
#valores_afectados
#matriz_control_cambios
#valores_afec_static_0_Error
#valores_afec_static_1_Error
#vb_and_mask
#vm_and_mask
#tensor_act
#values_to_run

In [ ]:
#Todo esto ha de ser una función que será llamada de la funcion apply round luego de inyectar los errores , pasando el tensor 
#original que llega con problemas , el tensor con los errores y la máscara . Con el fin de obtener el tensor actualizado con los valores
#redondeados .

In [26]:
#Se crea un tensor variable donde se guardarán los valores en cada ciclo que aún no se han encontrado la posición con error
#y hay que continuar analizando con mask>>1
# se cra un tensor con el valos de la máscara inicial la cual es 1000000000000000
def funcionRound(tensor,tensor_with_error):   
    
    values_to_run=tf.Variable(0)
    mask=tf.convert_to_tensor(32768)

#crea un tensor de (T and F),donde en T están los valores afectados por los errores inyectados
# esta matriz es la que controla si ya no hay cambios pendientes porque todos los valores han sido analizados y
#detiene el ciclo para no iterar innecesariamente

    valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)
    print('valores_afectados',valores_afectados)  

# Se extraen los valores que fueron afectados por los errores
   v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True))
print('valores afectados por los errores' ,v_m)

#valores que se deberian escribir
   v_b=tf.gather_nd(tensor_original,tf.where(valores_afectados==True))
print('valores que se deberian escribir' ,v_b)
#a=np.size(v_m)
# Creo una máscara de 0 del con la cantidad de errores afectados para ir llevando el contrl de los cambios 
#mask_0=tf.convert_to_tensor([0]*a)
   mask_0=tf.constant(0)
    print('mask_0',mask_0)

# inicializo los valores de comienzo y fin del ciclo teniendo en cuenta que pueden haber errores hasta la posición 15 de
# una palabra
   i=tf.constant(0)
    out=tf.constant(16)
    print(' out inicial',(out))
    print('i inicial', i)


#matriz_control(esta la cree pero al final no la uso luego que verifique todo puede ser eliminada sino hace falta)
#matriz_control=tf.equal(tensor_original,tensor_with_error)
    matriz_control_cambios=tf.equal(v_b,v_m)
    print('matriz_control_cambios',matriz_control_cambios)

# Tensores con los valores de errores en 0 y 1 que propiciaron cambios en el tensor original
    valores_afec_static_0_Error=tf.gather_nd(faults[:,0], tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
    valores_afec_static_1_Error=tf.gather_nd(faults[:,1], tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
    print('valores_afec_static_0_Error',valores_afec_static_0_Error)
    print('valores_afec_static_1_Error',valores_afec_static_1_Error)

    vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
    print('vb_and_mask',vb_and_mask)
    vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
    vm_and_mask
##### 'aquí actualizo el tensor con los nuevos valores de igual a mask'
    print('vm_and_mask',vm_and_mask)
#tf.gather_nd(v_b,tf.where(tf.equal(vb_and_mask,vb_and_mask)):
#a=np.size(v_m)

## la primera vez tensor_actu= tensor original creo esta variable para ir actualizando el tensor con los nuevos valores 
#generados con la técnica del redeondeo

   tensor_act=tensor_original 

valores_afectados tf.Tensor([ True  True  True], shape=(3,), dtype=bool)
valores afectados por los errores tf.Tensor([16422   102 33283], shape=(3,), dtype=int32)
valores que se deberian escribir tf.Tensor([   38    38 32769], shape=(3,), dtype=int32)
mask_0 tf.Tensor(0, shape=(), dtype=int32)
 out inicial tf.Tensor(16, shape=(), dtype=int32)
i inicial tf.Tensor(0, shape=(), dtype=int32)
matriz_control_cambios tf.Tensor([False False False], shape=(3,), dtype=bool)
valores_afec_static_0_Error tf.Tensor([49087 49087 65471], shape=(3,), dtype=int32)
valores_afec_static_1_Error tf.Tensor([16384    64   514], shape=(3,), dtype=int32)
vb_and_mask tf.Tensor([    0     0 32768], shape=(3,), dtype=int32)
vm_and_mask tf.Tensor([    0     0 32768], shape=(3,), dtype=int32)


In [14]:

#a:Matriz de True and False donde True son las posiciíones donde vb_and_mask y vm_and_mask son diferentes
#por tanto existe un error en la posición más significativa 
#b:Matriz de True and False donde True  es la posiciíon dende se cumple que (v_m>v_b ) 
#c: Indice donde la matriz a y b son verdaderas por tanto el error esta dado por un 1 
#values_to_change:los valores de v_b en las posiciones c que han se ser enviados a la función ErroraUno para su transformación
#val_afec_0_Error_to_change:Valores que corresponden los el error a 0 
#val_afec_1_Error_to_change:valores que corresponden con elerror a 1  Creo que ya no son necesarias 
#index: devuelve el valor de c en una variable numpy
#val:devuelve una lista de elementos que corresponden a los indices de valores afectados ==True que serán los indices a transformar
#error_a_0: Matriz de True and False donde True  es la posiciíon dende se cumple que (v_b>v_m )
#index_values_error_a_0:Indice donde la matriz a y error_a_0 que son verdaderas por tanto el error esta dado por un 0 
#val_0=devuelve una lista de elementos que corresponden a los indices de valores afectados ==True que serán los indices a transformar
#tensor_act: devuelve el tensor ya transormado con los elementos en las posiciones correspondientes donde se han encontrado los casos analizados


In [27]:
#def ErrorPosicionInicial(v_m,v_b):
#    b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
#    print('b Vm>Vb',b)
#    c=tf.where(tf.logical_and(a,b)==True)
#    print('c',c)
#    print('tamaño de c',tf.size(c))
#    print(' c Indices de los valores a transformar a 0',c)
#    values_to_change=tf.gather_nd(v_b,c)
#    print('mask_0 desdes update', mask_0)
#    index=c.numpy()
#    print('cnumpy',index)
#    print('valores_afectados',valores_afectados)
#    val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
#    print('val',val)
#    #mask_0=tf.add(mask_0,tf.size(val))
#    return(val)

In [28]:
#def TensorUpdatePosicionInicial(tensor_act,val,valores_afectados,mask_0): 
#    print('Dentro de TensorUpdatePosicionInicial')
#    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tf.convert_to_tensor([0]*np.size(val) ))
#    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
#    print('valores_afectados dentro de error 0 en primera posición',valores_afectados )
#    mask_0=tf.add(mask_0,tf.size(val))
#    print('tensor_act funciion a 0',tensor_act)
#    print('mask_0',mask_0)
#    print('valores_afectados',valores_afectados)
#    print('tensor_with_error',tensor_with_error)
#    #tensor_with_error_run= tf.gather_nd(tensor_with_error,tf.where(valores_afectados ==True))   
#    v_b=tf.gather_nd(tensor_act,tf.where(valores_afectados==True))
#    print('v_b',v_b)
#    v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True) )
#    print('v_m',v_m)
#    valores_afec_static_0_Error=tf.gather_nd(faults[:,0],tf.where(valores_afectados==True))
#    print('valores_afec_static_0_Error', valores_afec_static_0_Error)
#    valores_afec_static_1_Error=tf.gather_nd(faults[:,1],tf.where(valores_afectados==True))
#    print('valores_afec_static_1_Error', valores_afec_static_0_Error)
#    return tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error

In [29]:
#def TensorUpdateCiclo(error,v_b,c,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0):       
#    values_to_change=tf.gather_nd(v_b,c)
#    val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
#    print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
#    val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
#    print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
#    print('values_to_change',values_to_change)
#    if error==1:
#        tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
#        print('tensor_round',tensor_round)
#    if error==0:
#        tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
#        print('tensor_round',tensor_round)
#    index=c.numpy()
#    print('indice a camniar tensor act',index)
#    print('valores_afectados',valores_afectados)
#    val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
#    print('val',val)
#    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
#    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
#    mask_0=tf.add(mask_0,tf.size(val))
#    print('mask_0 desdes update', mask_0)
#    print('tensor tensor_act',tensor_act)
#    print('valores_afectados update',valores_afectados) 
#    #del tensor_round
#    print('tensor_round',tensor_round)
#    return tensor_act,valores_afectados,mask_0
#    

In [12]:
## casos para el signo tanto para 0 en la primera posición como q

In [30]:
    a=tf.math.not_equal(vb_and_mask,vm_and_mask)
    print('estoy calculando a y definiendo xq esta dado el error')
    print('a vb_and_mask!=vm_and_mask',a)
    print('mask',mask)
    ##++ Hasta aqui es una función que ppuede ser llamada pues se repite 
    val=ErrorPosicionInicial(v_b,v_m)
    print('val',val)
    
    #### agraegado para si se encuentra error a 0 en la posición más significativa  esto 
    val_0=ErrorPosicionInicial(v_m,v_b)
    print('val_0',val_0)
    
    #tf.cond( np.size(val_0)>0:, f1, f2)
    
    if np.size(val_0)>0:
        tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error=TensorUpdatePosicionInicial(tensor_act,val_0,valores_afectados,mask_0)
        
    ## entra a este if cuando hay error a 1 en la posición más significativa    
    if np.size(val)> 0: 
        tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error=TensorUpdatePosicionInicial(tensor_act,val,valores_afectados,mask_0)
        
        
    print('tensor_act',tensor_act)
    print('valores_afectados',valores_afectados)
    print('tensor_with_error',tensor_with_error)
    print('v_b',v_b)
    print('v_m',v_m)
    print('valores_afec_static_0_Error', valores_afec_static_0_Error)
    print('valores_afec_static_1_Error',valores_afec_static_1_Error)
    mask=tf.bitwise.right_shift(mask,1)

estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([False False False], shape=(3,), dtype=bool)
mask tf.Tensor(32768, shape=(), dtype=int32)
b Vm>Vb tf.Tensor([False False False], shape=(3,), dtype=bool)
c tf.Tensor([], shape=(0, 1), dtype=int64)
tamaño de c tf.Tensor(0, shape=(), dtype=int32)
 c Indices de los valores a transformar a 0 tf.Tensor([], shape=(0, 1), dtype=int64)
mask_0 desdes update tf.Tensor(0, shape=(), dtype=int32)
cnumpy []
valores_afectados tf.Tensor([ True  True  True], shape=(3,), dtype=bool)
val tf.Tensor([], shape=(0, 1), dtype=int64)
val tf.Tensor([], shape=(0, 1), dtype=int64)
b Vm>Vb tf.Tensor([ True  True  True], shape=(3,), dtype=bool)
c tf.Tensor([], shape=(0, 1), dtype=int64)
tamaño de c tf.Tensor(0, shape=(), dtype=int32)
 c Indices de los valores a transformar a 0 tf.Tensor([], shape=(0, 1), dtype=int64)
mask_0 desdes update tf.Tensor(0, shape=(), dtype=int32)
cnumpy []
valores_afectados tf.Tensor([ True  True  Tr

## modificarlo porque en el caso donde hay 0 en la posición 1 no lo estaba contemplando

In [ ]:
#values_to_run_with_error
#valores_afec_static_0_Error_to_run
#valores_afec_static_0_Error_to_run
#values_to_change

In [31]:
    def while_body(i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error):
        print('dentro del body')
        print('i',i)
        print('out que llega al body', out)
        print('tensor_act',tensor_act)
        print('tensor_original',tensor_original)
        print('mask',mask)
        print('mask_0',mask_0)
        print('v_m',v_m)
        print('v_b',v_b)
        print('tensor_with_error',tensor_with_error)
        #hay_cambios=tf.math.not_equal(tensor_original,tensor_act)
        #print('hay_cambios',hay_cambios)
        #stop_condition=tf.experimental.numpy.any(valores_afectados)
        #if stop_condition==False:
            
        ## Este if lo cree por si hay cambios antes de llegar al ciclo por haber errores e la posición 15 pero quizás ya o sea
        # necesraio analizar luego de experimentar  si mask_0 llega con valores a 1 es porque se hicieron cambios y se deben
        #analizar los btw con los valores resttantes sino todo se mantiene con los valores originales de entrada.
        if tf.reduce_sum(mask_0)>0:
            print('entre en condicion mask_0>0')
            print('valores donde debo correr el mask')
            
            vb_and_mask=tf.bitwise.bitwise_and(v_b,mask)
            print('vb_and_mask',vb_and_mask)
            vm_and_mask=tf.bitwise.bitwise_and(v_m,mask)
            print('vm_and_mask',vm_and_mask)  
        else:
            
             ### esta estructura cambiaria porque ya tendre solo los valores con problemas, tomaria valores de tensores variables 
             #inicializados en 0 hasta que tome sus valores luego del recorrido por el body
             ## el tensor control tomara un papel aca porque luego actualizaria el tensor_actu en los posiciones donde 
                 #tensorcontrol sea false mas otra condicion mas o debo ir comparando el tensor actu con el original y envio los dos
            #valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)## inicialmente no hara nada xq son iguales
            print('entre en el else')
            vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
            print('vb_and_mask',vb_and_mask)
            vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
            print('vm_and_mask',vm_and_mask) 
            print('v_b',v_b)
            print('v_m',v_m)
            
        a=tf.math.not_equal(vb_and_mask,vm_and_mask)
        print('estoy calculando a y definiendo xq esta dado el error')
        print('a vb_and_mask!=vm_and_mask',a)
        values_to_run=tf.gather_nd(v_b,tf.where(a==False))
        values_to_run_with_error=tf.gather_nd(v_m,tf.where(a==False))
        valores_afec_static_0_Error_to_run=tf.gather_nd(valores_afec_static_0_Error,tf.where(a==False))
        print('valores_afec_static_0_Error_to_run',valores_afec_static_0_Error_to_run)
        valores_afec_static_1_Error_to_run=tf.gather_nd(valores_afec_static_1_Error,tf.where(a==False))
        print('valores_afec_static_1_Error_to_run',valores_afec_static_1_Error_to_run)
        print('values_to_run', values_to_run)
        print(' values_to_run_with_error', values_to_run_with_error)
        #valores_afectados_to_run= valores_afectados
        #u=tf.size(values_to_run)>0
        
        
        b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
        print('b Vm>Vb',b)
        c=tf.where(tf.logical_and(a,b)==True)
        #c=tf.where(tf.equal(a,b))
        print('tamaño de c',tf.size(c))
        print(' c Indices de los valores a transformar con la variante error a 1',c)
        
        error_a_0=tf.math.greater(v_b,v_m)
        print('error_a_0',error_a_0)
        index_values_error_a_0=tf.where(tf.logical_and(a,error_a_0)==True)
        #index_values_error_a_0=tf.where(error_a_0==True)
        print('index_values_error_a_0',index_values_error_a_0)
        print('vb',v_b)
        
            
       
        if tf.size(c)>0:
            error=1
            tensor_act,valores_afectados,mask_0= TensorUpdateCiclo(error,v_b,c,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0)
            #values_to_change=tf.gather_nd(v_b,c)
            #val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
            #print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
            #val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
            #print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
            #print('values_to_change',values_to_change)
            #tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
            #print('tensor_round',tensor_round)
            #index=c.numpy()
            #print('indice a camniar tensor act',index)
            #print('valores_afectados',valores_afectados)
            #val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
            #print('val',val)
            #tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
            #valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
            #mask_0=tf.add(mask_0,tf.size(val))
            print('mask_0 desdes update', mask_0)
            print('tensor tensor_act',tensor_act)
            print('valores_afectados update',valores_afectados)                                              
        
        
            
            
            
        
            
        ##no es el que esta en mask en esse momento, retorno new mask, new_mask_0, tensor_act
        if tf.size(index_values_error_a_0)>0: 
            error=0
            print('dentro detf.size(error_a_0)>0 ')
            tensor_act,valores_afectados,mask_0= TensorUpdateCiclo(error,v_b,index_values_error_a_0,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0)
            #out=tf.constant(tf.math.subtract(out,tf.size(error_a_0)))
            #print('out',out)
            #mask_0=tf.tensor_scatter_nd_update(mask_0,error_a_0,tf.convert_to_tensor([1]*np.size(error_a_0)))
            print()
            #out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
            #values_to_change=tf.gather_nd(v_b,index_values_error_a_0)
            #val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,index_values_error_a_0)
            #print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
            #val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,index_values_error_a_0)
            #print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
            #print('values_to_change',values_to_change)
            #tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
            #index_0=index_values_error_a_0.numpy()
            #print('index_0',index_0)
            #print('valores_afectados',valores_afectados)
            #val_0=tf.gather_nd(tf.where(valores_afectados),index_0)## tomo los indices del tensor original qu ese cambiaran
            #print('val_0',val_0)
            #
            #tensor_act = tf.tensor_scatter_nd_update(tensor_act, val_0, tensor_round )
            #valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val_0,tf.convert_to_tensor([False]*np.size(val_0)))
            ##print('tensor_round en el body',tensor_round)
            print('tensor tensor_act',tensor_act)
            print('valores_afectados update',valores_afectados)        
            #mask_0=tf.add(mask_0,tf.size(val_0))
            print('mask_0 desdes update', mask_0)
        
            
            
            
        if tf.size(values_to_run)>0:
            #mask=mask>>1
            mask=tf.bitwise.right_shift(mask,1)
            print('dentro del if values to run',values_to_run)
            print('mask',mask)
            v_b=values_to_run
            v_m=values_to_run_with_error 
            valores_afec_static_0_Error=valores_afec_static_0_Error_to_run
            valores_afec_static_1_Error=valores_afec_static_1_Error_to_run
            print('v_b',v_b)
            print('v_m',v_m)
            print('valores_afec_static_0_Error',valores_afec_static_0_Error)
            print('valores_afec_static_1_Error',valores_afec_static_1_Error)
            
        
        
        if tf.experimental.numpy.any(valores_afectados)==False:
            #tensor=tf.bitwise.bitwise_and(tensor_act,val_afec_0_Error_to_change)
            #print('n_v_b_1', tensor)
            #tensor_round=tf.bitwise.bitwise_or(tensor,val_afec_1_Error_to_change)
            print('valores_afectados',valores_afectados)
            print('tensor_act',tensor_act)
            i=16
            
        print('i',i) 
        
        
        return (i+1),out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error ## retorno los valores que aun no se han modificado por ninguna d elas variantes puesto que el bit con error     
        
    i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error= tf.while_loop(
        lambda i, *_ : tf.less(i,out),  # condición -> revisar cada bit
         while_body,  # aqui todo lo que tengas que hacer de comparar los vectores (que estos los puedes definir por fuera
                     # o pasarlos a la misma función de while_body y no modificarlos)
        (i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error)  # valores iniciales
    )   
    
    

dentro del body
i tf.Tensor(0, shape=(), dtype=int32)
out que llega al body tf.Tensor(16, shape=(), dtype=int32)
tensor_act tf.Tensor([   38    38 32769], shape=(3,), dtype=int32)
tensor_original tf.Tensor([   38    38 32769], shape=(3,), dtype=int32)
mask tf.Tensor(16384, shape=(), dtype=int32)
mask_0 tf.Tensor(0, shape=(), dtype=int32)
v_m tf.Tensor([16422   102 33283], shape=(3,), dtype=int32)
v_b tf.Tensor([   38    38 32769], shape=(3,), dtype=int32)
tensor_with_error tf.Tensor([16422   102 33283], shape=(3,), dtype=int32)
entre en el else
vb_and_mask tf.Tensor([0 0 0], shape=(3,), dtype=int32)
vm_and_mask tf.Tensor([16384     0     0], shape=(3,), dtype=int32)
v_b tf.Tensor([   38    38 32769], shape=(3,), dtype=int32)
v_m tf.Tensor([16422   102 33283], shape=(3,), dtype=int32)
estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([ True False False], shape=(3,), dtype=bool)
valores_afec_static_0_Error_to_run tf.Tensor([49087 65471], shape=(2,